## Data Preparation

In [ ]:
import os
Root = "/kaggle/input/ravdess/"
os.chdir(Root)

# Importing Libraries

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

##  <center> 1. Ravdess Dataframe <center>


In [ ]:
ravdess_directory_list = os.listdir(Root)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Root + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Root + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

In [ ]:
# creating Dataframe using ravdees dataframes we created so far.
data_path = pd.concat([Ravdess_df], axis = 0)
data_path.to_csv("/kaggle/working/data_path.csv",index=False)
# data_path.head()

In [ ]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate=rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sr=22050, n_steps=2):
    return librosa.effects.pitch_shift(data, sr=sr, n_steps=n_steps)

# taking any example and checking for techniques.
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)

In [ ]:
def extract_features(data):

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = librosa.feature.chroma_stft(S=stft, sr=sample_rate, n_chroma=64)
    # If the resulting Chroma_stft matrix has fewer frames than n_frames, you can pad it.
    if chroma_stft.shape[1] < 108:
        pad_width = 108 - chroma_stft.shape[1]
        chroma_stft = np.pad(chroma_stft, ((0, 0), (0, pad_width)), mode='constant')

    # If the resulting Chroma_stft matrix has more frames than n_frames, you can truncate it.
    if chroma_stft.shape[1] > 108:
        chroma_stft = chroma_stft[:, :108]
    
    
    # MFCC
    mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=64)
    # If the resulting MFCC matrix has fewer frames than n_frames, you can pad it.
    if mfccs.shape[1] < 108:
        pad_width = 108 - mfccs.shape[1]
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')

    # If the resulting MFCC matrix has more frames than n_frames, you can truncate it.
    if mfccs.shape[1] > 108:
        mfccs = mfccs[:, :108]
          
    # MelSpectogram
    mel = librosa.feature.melspectrogram(y=data, sr=sample_rate, n_mels=64)
    # If the resulting MelSpectogram matrix has fewer frames than n_frames, you can pad it.
    if mel.shape[1] < 108:
        pad_width = 108 - mel.shape[1]
        mel = np.pad(mel, ((0, 0), (0, pad_width)), mode='constant')

    # If the resulting MelSpectogram matrix has more frames than n_frames, you can truncate it.
    if mel.shape[1] > 108:
        mel = mel[:, :108]
        
    result = np.stack((chroma_stft, mfccs,mel)) # stacking horizontally 
    
    return result

def get_features(path):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # without augmentation
    res1 = extract_features(data) # 13
    result = np.array(res1) # (13, )

    # data with noise
    noise_data = noise(data)
    res2 = extract_features(noise_data) # (13, )
    result = np.vstack((result, res2)) # stacking vertically (2, 13)
    
    result = result.reshape(2, 64, 108, 3)
    
    return result

In [ ]:
X, Y = [], []

for path, emotion in zip(data_path.Path, data_path.Emotions):
    feature = get_features(path)

    for ele in feature:
        X.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y.append(emotion)

In [ ]:
print(feature.shape)
print(ele.shape)

In [ ]:
X = np.array(X)
Y = np.array(Y)

print(X.shape, Y.shape, len(X), len(Y))

In [ ]:
num_samples = len(X)
num_frequency_bins = X[0].shape[0]
num_time_steps = X[0].shape[1]
num_fea_type = X[0].shape[2]

print("Number of samples:", num_samples)
print("Number of frequency bins:", num_frequency_bins)
print("Number of time steps:", num_time_steps)
print("Number of feature_method:", num_fea_type)

In [ ]:
# Flatten the 3D feature array into 2D
X_flattened = X.reshape(X.shape[0], -1)

Features = pd.DataFrame(X_flattened)
Features['labels'] = Y
Features.to_csv('/kaggle/working/features.csv', index=False)
Features.head()

## Data Preparation

- As of now we have extracted the data, now we need to normalize and split our data for training and testing.

In [ ]:
# X = Features.iloc[: ,:-1].values
X = np.array(Features.iloc[:, :-1].values).reshape(num_samples, num_frequency_bins, num_time_steps, num_fea_type)
Y = Features['labels'].values

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.2, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

# Reshape the data to 2D
x_train_2d = x_train.reshape(x_train.shape[0], -1)
x_test_2d = x_test.reshape(x_test.shape[0], -1)

# Apply StandardScaler to the 2D data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_2d)
x_test_scaled = scaler.transform(x_test_2d)

# Reshape the scaled data back to 3D
x_train = x_train_scaled.reshape(x_train.shape[0], num_frequency_bins, num_time_steps, num_fea_type)
x_test = x_test_scaled.reshape(x_test.shape[0], num_frequency_bins, num_time_steps, num_fea_type)

x_train.shape, y_train.shape, x_test.shape, y_test.shape


## Modelling

In [ ]:
import keras
from keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
import numpy as np

# Define the model
model = Sequential()

# First Conv3D layer
model.add(Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu', input_shape=(num_frequency_bins, num_time_steps, num_fea_type, 1)))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 1)))

# Second Conv3D layer
model.add(Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 1), strides=(2, 2, 1)))

# Second Conv3D layer
model.add(Conv3D(128, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 1), strides=(2, 2, 1)))

# Third Conv3D layer
model.add(Conv3D(256, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 1), strides=(2, 2, 1)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(1000, activation='relu'))
# model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001)
history=model.fit(x_train, y_train, batch_size=64, epochs=100, validation_data=(x_test, y_test), callbacks=[rlrp])

In [ ]:
import matplotlib.pyplot as plt

print("Accuracy of our model on test data : " , model.evaluate(x_test,y_test)[1]*100 , "%")

epochs = [i for i in range(100)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(20,6)
ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()

In [ ]:
# predicting on test data.
pred_test = model.predict(x_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

In [ ]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(10)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


cm = confusion_matrix(y_test, y_pred)
# plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = [i for i in encoder.categories_] , columns = [i for i in encoder.categories_])
sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, zero_division=0))